In [17]:
import os
from dotenv import load_dotenv

In [18]:
import pdfplumber

class Model():

    def __init__(self, printAnswers=False):
        self.setupModel()

        self.printAnswers = printAnswers

        self.systemPrompt = """
Din roll: Sammanfattare av texter på ett enkelt och förståeligt sätt.

Ditt syfte: Hjälpa människor förstå svårlästa texter, som exempelvis myndighetsbeslut, kontrakt, avier, eller fakturor, genom att översätta dem till klartext. Målet är att användaren enkelt ska kunna förstå vad som står i dokumentet och vad som behöver göras.

Metod: Använd följande steg för att sammanfatta texten:

1. Identifiera det viktigaste: Fokusera på de mest relevanta delarna av texten som användaren behöver veta.
2. Sammanfatta i korta och tydliga meningar: Undvik onödiga detaljer och använd enkel svenska.
3. Specificera viktiga detaljer: Om det finns siffror, belopp, datum eller liknande som är viktiga, lyft fram dessa tydligt.
4. Sammanfatta vad som behöver göras: Ge en kortfattad översikt av vad användaren behöver göra eller vara medveten om.


Din uppgift: Användaren kommer att skicka en text som representerar den information som finns i dokumentet de vill ha hjälp med att förstå. Din uppgift är att sammanfatta denna text enligt instruktionerna ovan.

Svara alltid på svenska.

Du sammanfattar texter som användaren skickar. 
Syftet är att översätta svårlästa texter som exempelvis myndighetsbeslut, 
kontrakt eller avier/fakturor till klartext som är lättförståelig och tydlig, 
framförallt om det är något som användaren behöver göra själv, en så kallad "action required" (exempelvis betala en faktura).

Texten som användaren kommer skicka är PDFer som har blivit omformaterade till en lång sträng. 
Detta kan leda till att viss positionell information försvinner, eller att olika textstycken 
kommer i en annan ordning än vad som är tänkt.
Därefter kommer användaren skicka ett antal flervalsfrågor som du ska svara på baserat på innehållet i PDFen. Frågorna kommer ha alternativ a, b, c, d, e, och f.

VIKTIGT: Svara endast med den bokstav som motsvarar det korrekta svaret. Skriv ingenting annat än den korrekta bokstaven.
"""

    def setupModel(self):
        pass

    def getModelAnswer(self, prompt):
        pass

    def extract_text_from_pdf(self, pdf):
        text = ""
        with pdfplumber.open("Documents/" + pdf) as pdf_file:
            for page in pdf_file.pages:
                text += page.extract_text()
        return text

In [19]:
class Evaluator():

    # Evaluates a given model on the following documents:
    #   1. Hyresavi
    #   2. Om krisen eller kriget kommer
    #   3. 

    def __init__(self, model):
        self.model = model
        

    def evaluate_all(self, model: Model) -> None:
        self.evaluate_hyresavi(model)
        self.evaluate_hyresavi2(model)
        self.evaluate_krisbrochure(model)
        self.evaluate_Askatt(model)
        self.evaluate_deklaration(model)
        self.evaluate_FMHCovid(model)
        self.evaluate_hovratt(model)

    def count_correct_answers(self, answers):
        correct = 0
        for key in answers:
            if answers[key]:
                correct += 1
        return correct

    def evaluate_hyresavi(self, model: Model) -> None:
        # Här vill vi skicka ett antal flervalsfrågor till modellen och be den svara på dem 
        answers = {}

        # Fråga 1: Vad är OCR nummret?        
        answer = model.getModelAnswer("""
                                      Vad är OCR nummret?
                                      a. 407025
                                      b. 5954-0146
                                      c. 317925899
                                      d. 123456789
                                      e. 6303-7105-026
                                      f. 4337,00
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[1] = True
        else:
            answers[1] = False

        # Fråga 2: Vad är beloppet som ska betalas?
        answer = model.getModelAnswer("""
                                      Vad är det totala beloppet som ska betalas (kr)?
                                      a. 407025
                                      b. 4170,00
                                      c. 317925899
                                      d. 167,00
                                      e. 6303-7105-026
                                      f. 4337,00
                                      """, "Hyresavi.pdf")
        
        if model.printAnswers: print(answer, "(rätt: f)")
        if answer == "f":
            answers[2] = True
        else:
            answers[2] = False

        # Fråga 3: När ska betalningen vara gjord?
        answer = model.getModelAnswer("""
                                      När ska betalningen vara gjord?
                                      a. 2024-12-31
                                      b. 2023-04-12
                                      c. 2024-03-31
                                      d. 2024-11-11
                                      e. 2024-02-29
                                      f. 2024-01-01
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[3] = True
        else:
            answers[3] = False
        
        # Fråga 4: Till vilket bank-giro ska avin betalas?

        answer = model.getModelAnswer("""
                                        Till vilket bank-giro ska avin betalas?
                                        a. 407025
                                        b. 5954-0146
                                        c. 317925899
                                        d. 123456789
                                        e. 6303-7105-026
                                        f. 4337,00
                                        """, "Hyresavi.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False
        
        # Rättning
        correct = self.count_correct_answers(answers)
        
        print(f"-- Hyresavi: \t\t{correct}/4 rätt")
        print()

    def evaluate_hyresavi2(self, model: Model)-> None:
        answers = {}

        # Fråga 1: Vilket belopp ska betalas för hyra av bostad för perioden 2024-04-01 till 2024-04-30?
        answer = model.getModelAnswer("""
                                      Vilket belopp ska betalas för hyra av bostad för perioden 2024-04-01 till 2024-04-30?
                                      a. 4170,00 SEK
                                      b. 4337,00 SEK
                                      c. 167,00 SEK
                                      d. 6303,00 SEK
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[1] = True
        else:
            answers[1] = False

        # Fråga 2: Vad är OCR-numret som ska anges vid betalning?
        answer = model.getModelAnswer("""
                                      Vad är OCR-numret som ska anges vid betalning?
                                      a. 5954-0146
                                      b. 802003-2887
                                      c. 317925899
                                      d. 6303-7105-026
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[2] = True
        else:
            answers[2] = False

        # Fråga 3: Vad är det totala beloppet som ska betalas?
        answer = model.getModelAnswer("""
                                      Vad är det totala beloppet som ska betalas?
                                      a. 4170,00 SEK
                                      b. 4337,00 SEK
                                      c. 167,00 SEK
                                      d. 4537,00 SEK
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[3] = True
        else:
            answers[3] = False

        # Fråga 4: Vilken organisation är betalningsmottagare?
        answer = model.getModelAnswer("""
                                      Vilken organisation är betalningsmottagare?
                                      a. Isak Nordgren
                                      b. Stockholms Studentbostäder SSSB
                                      c. SSAB
                                      d. Kungshamra 71
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False

        # Fråga 5: Vad inkluderar specifikationen för perioden 2024-04-01 till 2024-04-30?
        answer = model.getModelAnswer("""
                                      Vad inkluderar specifikationen för perioden 2024-04-01 till 2024-04-30?
                                      a. Hyra bostad och internetanslutning
                                      b. Hyra bostad och vatten
                                      c. Hyra bostad och el
                                      d. Hyra bostad och parkering
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[5] = True
        else:
            answers[5] = False

        # Fråga 6: Vilket är det sista betalningsdatumet som anges på hyresavin?
        answer = model.getModelAnswer("""
                                      Vilket är det sista betalningsdatumet som anges på hyresavin?
                                      a. 2024-03-31
                                      b. 2024-03-13
                                      c. 2024-04-01
                                      d. 2024-04-30
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[6] = True
        else:
            answers[6] = False

        # Fråga 7: Vilket är kundnumret som anges på hyresavin?
        answer = model.getModelAnswer("""
                                      Vilket är kundnumret som anges på hyresavin?
                                      a. 6303-7105-026
                                      b. 802003-2887
                                      c. 407025
                                      d. 317925899
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[7] = True
        else:
            answers[7] = False

        # Fråga 8: Vilken adress har betalningsmottagaren?
        answer = model.getModelAnswer("""
                                      Vilken adress har betalningsmottagaren?
                                      a. Kungshamra 71 / 1312, 170 70 Solna
                                      b. Box 19608, 104 32 Stockholm
                                      c. information@sssb.se
                                      d. Tel: 08-458 10 10
                                      """, "Hyresavi.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[8] = True
        else:
            answers[8] = False

        # Rättning
        correct = self.count_correct_answers(answers)
        
        print(f"-- Hyresavi 2: \t\t{correct}/8 rätt")
        print()

    def evaluate_krisbrochure(self, model: Model) -> None:
        # Här vill vi skicka ett antal flervalsfrågor till modellen och be den svara på dem 
        answers = {}
    
        # Fråga 1: Vad är syftet med broschyren "Om krisen eller kriget kommer"?
        answer = model.getModelAnswer("""
                                      Vad är syftet med broschyren "Om krisen eller kriget kommer"?
                                      a. Att informera om Sveriges kulturarv
                                      b. Att underhålla med intressanta berättelser
                                      c. Att förbereda invånare på kriser och krigssituationer
                                      d. Att sälja säkerhetsutrustning
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[1] = True
        else:
            answers[1] = False
    
        # Fråga 2: Vad ska du göra om du hör signalen för "Viktigt meddelande till allmänheten"?
        answer = model.getModelAnswer("""
                                      Vad ska du göra om du hör signalen för "Viktigt meddelande till allmänheten"?
                                      a. Ignorera den och fortsätta som vanligt
                                      b. Gå inomhus, stänga fönster och dörrar, och lyssna på Sveriges Radio P4
                                      c. Samlas utomhus med grannar och diskutera situationen
                                      d. Ringa polisen för mer information
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[2] = True
        else:
            answers[2] = False
    
        # Fråga 3: Vilka är de två huvudsakliga komponenterna i Sveriges totalförsvar?
        answer = model.getModelAnswer("""
                                      Vilka är de två huvudsakliga komponenterna i Sveriges totalförsvar?
                                      a. Militärt försvar och civilt försvar
                                      b. Polisen och sjukvården
                                      c. Brandkåren och Försvarsmakten
                                      d. Regeringen och riksdagen
                                      """, "Kris.pdf")
        
        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[3] = True
        else:
            answers[3] = False
    
        # Fråga 4: Vilken typ av mat rekommenderas att ha hemma för krisberedskap?
        answer = model.getModelAnswer("""
                                      Vilken typ av mat rekommenderas att ha hemma för krisberedskap?
                                      a. Färska frukter och grönsaker
                                      b. Hållbar mat som kräver liten mängd vatten eller kan ätas utan tillagning
                                      c. Mat från snabbmatsrestauranger
                                      d. Färdiglagad mat från catering
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False
    
        # Fråga 5: Vad innebär "totalförsvarsplikt" i Sverige?
        answer = model.getModelAnswer("""
                                      Vad innebär "totalförsvarsplikt" i Sverige?
                                      a. Endast militärer måste bidra vid krig
                                      b. Alla mellan 16 och 70 år kan kallas in för att hjälpa till vid krigsfara och krig
                                      c. Endast män måste delta i försvaret
                                      d. Endast frivilliga behöver delta vid krigsfara
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[5] = True
        else:
            answers[5] = False
    
        # Fråga 6: Vad bör du göra om du upplever en falsk informationskampanj?
        answer = model.getModelAnswer("""
                                      Vad bör du göra om du upplever en falsk informationskampanj?
                                      a. Ignorera informationen
                                      b. Sprida informationen vidare utan verifiering
                                      c. Vara källkritisk och använda flera trovärdiga källor för att verifiera informationen
                                      d. Lita på alla rykten du hör
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[6] = True
        else:
            answers[6] = False
    
        # Fråga 7: Vad innebär beredskapslarmet?
        answer = model.getModelAnswer("""
                                      Vad innebär beredskapslarmet?
                                      a. Att du ska gå ut och ta reda på mer information
                                      b. Att det råder omedelbar krigsfara eller att landet är i krig
                                      c. Att en naturkatastrof är nära förestående
                                      d. Att du ska kontakta dina släktingar omedelbart
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[7] = True
        else:
            answers[7] = False
    
        # Fråga 8: Vad är en av de första åtgärderna du ska vidta vid ett terrorattentat?
        answer = model.getModelAnswer("""
                                      Vad är en av de första åtgärderna du ska vidta vid ett terrorattentat?
                                      a. Larma polisen via 112 och meddela om du ser något viktigt
                                      b. Ta bilder och dela på sociala medier
                                      c. Samlas i stora grupper för att diskutera händelsen
                                      d. Fortsätta med dina dagliga aktiviteter som vanligt
                                      """, "Kris.pdf")
    
        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[8] = True
        else:
            answers[8] = False
        
        # Rättning
        correct = self.count_correct_answers(answers)
        
        print(f"-- Krisbrochure: \t{correct}/8 rätt")
        print()

    def evaluate_Askatt(self, model: Model) -> None:
        answers = {}

        # Fråga 1: Vad ska bifogas anmälan om preliminär A-skatt om du kommer från ett land utanför EES-området eller Schweiz?
        answer = model.getModelAnswer("""
                                      Vad ska bifogas anmälan om preliminär A-skatt om du kommer från ett land utanför EES-området eller Schweiz?
                                      a. Kopia av LMA-kort
                                      b. Kopia av arbetstillstånd
                                      c. Kopia av pass
                                      d. Kopia av anställningsavtal
                                      """, "A-skatt.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[1] = True
        else:
            answers[1] = False

        # Fråga 2: Vilken typ av handling ska styrka din identitet om du saknar svenskt person-/samordningsnummer?
        answer = model.getModelAnswer("""
                                      Vilken typ av handling ska styrka din identitet om du saknar svenskt person-/samordningsnummer?
                                      a. Kopia av pass eller nationellt ID-kort
                                      b. Kopia av arbetstillstånd
                                      c. Kopia av LMA-kort
                                      d. Kopia av anställningsavtal
                                      """, "A-skatt.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[2] = True
        else:
            answers[2] = False

        # Fråga 3: Vad används för att registrera personuppgifter i skatteregistret?
        answer = model.getModelAnswer("""
                                      Vad används för att registrera personuppgifter i skatteregistret?
                                      a. Kopia av LMA-kort
                                      b. Kopia av pass
                                      c. Tjänsteanteckningar
                                      d. Skatteverkets anteckningar
                                      """, "A-skatt.pdf")

        if model.printAnswers: print(answer, "(rätt: d)")
        if answer == "d":
            answers[3] = True
        else:
            answers[3] = False

        # Fråga 4: Vilken information om arbetsgivaren ska inkluderas i anmälan?
        answer = model.getModelAnswer("""
                                      Vilken information om arbetsgivaren ska inkluderas i anmälan?
                                      a. Namn och organisationsnummer
                                      b. Namn och medborgarskap
                                      c. Namn och lägenhetsnummer
                                      d. Namn och inkomst
                                      """, "A-skatt.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[4] = True
        else:
            answers[4] = False

        # Fråga 5: Vad är syftet med att bifoga en kopia på LMA-kort?
        answer = model.getModelAnswer("""
                                      Vad är syftet med att bifoga en kopia på LMA-kort?
                                      a. Det styrker din identitet
                                      b. Det styrker din inkomst
                                      c. Det styrker ditt medborgarskap
                                      d. Det styrker att du har rätt att vistas i Sverige
                                      """, "A-skatt.pdf")

        if model.printAnswers: print(answer, "(rätt: d)")
        if answer == "d":
            answers[5] = True
        else:
            answers[5] = False

        # Rättning
        correct = self.count_correct_answers(answers)

        print(f"-- A-skatt: \t\t{correct}/5 rätt")
        print()

    def evaluate_deklaration(self, model: Model) -> None:
        answers = {}

        # Fråga 1: Vilka metoder kan användas för att deklarera om du bara ska godkänna deklarationen utan att göra några ändringar?
        answer = model.getModelAnswer("""
        Vilka metoder kan användas för att deklarera om du bara ska godkänna deklarationen utan att göra några ändringar?
        a) I appen Skatteverket, via e-tjänsten, genom att skicka ett sms, genom att ringa eller på pappersblanketten.
        b) Endast i appen Skatteverket och på pappersblanketten.
        c) Via e-tjänsten och genom att ringa.
        d) Genom att ringa eller skicka ett fax.
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[1] = True
        else:
            answers[1] = False

        # Fråga 2: Vilken av följande metoder kan inte användas om du ska begära avdrag för resor till och från arbetet utan att göra några andra ändringar?
        answer = model.getModelAnswer("""
        Vilken av följande metoder kan inte användas om du ska begära avdrag för resor till och från arbetet utan att göra några andra ändringar?
        a) I appen Skatteverket
        b) Via e-tjänsten Inkomstdeklaration 1
        c) På pappersblanketten
        d) Genom att ringa
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: d)")
        if answer == "d":
            answers[2] = True
        else:
            answers[2] = False

        # Fråga 3: Vad behöver du använda för att logga in och deklarera i appen Skatteverket?
        answer = model.getModelAnswer("""
        Vad behöver du använda för att logga in och deklarera i appen Skatteverket?
        a) Ansiktsigenkänning eller fingeravtryck
        b) Din e-legitimation eller koder från informationsbrevet
        c) Bank-ID eller pass
        d) Endast personnummer och telefonnummer
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[3] = True
        else:
            answers[3] = False

        # Fråga 4: Vilken information behöver du för att skicka ett sms och godkänna din deklaration?
        answer = model.getModelAnswer("""
        Vilken information behöver du för att skicka ett sms och godkänna din deklaration?
        a) Personnummer och e-legitimation
        b) Personnummer och kod för underskrift
        c) Endast personnummer
        d) Endast kod för underskrift
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False

        # Fråga 5: Vad måste du göra om du upptäcker en felaktig uppgift i din deklaration?
        answer = model.getModelAnswer("""
        Vad måste du göra om du upptäcker en felaktig uppgift i din deklaration?
        a) Ändra uppgiften själv i e-tjänsten eller på pappersblanketten
        b) Kontakta uppgiftslämnaren och Skatteverket
        c) Skicka en ny deklaration
        d) Ignorera felet om det är litet
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[5] = True
        else:
            answers[5] = False

        # Fråga 6: Om du behöver lämna en bilaga till deklarationen, vad kan du använda?
        answer = model.getModelAnswer("""
        Om du behöver lämna en bilaga till deklarationen, vad kan du använda?
        a) Enbart pappersblanketten
        b) E-tjänsten eller importera bilagan via fil
        c) En fax till Skatteverket
        d) Appen Skatteverket
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[6] = True
        else:
            answers[6] = False

        # Fråga 7: Vilket nummer ska du skicka ditt sms till om du deklarerar från utlandet?
        answer = model.getModelAnswer("""
        Vilket nummer ska du skicka ditt sms till om du deklarerar från utlandet?
        a) +46 71144
        b) 71144 utan landskod
        c) +46 8 764 92 40
        d) +46 20 567 100
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[7] = True
        else:
            answers[7] = False

        # Fråga 8: Vad ska du göra om du vill ändra något i din deklaration efter att den redan skickats in och du fått slutskattebeskedet?
        answer = model.getModelAnswer("""
        Vad ska du göra om du vill ändra något i din deklaration efter att den redan skickats in och du fått slutskattebeskedet?
        a) Göra ändringen direkt i e-tjänsten
        b) Begära en omprövning genom att skicka ett brev till Skatteverket
        c) Inget, ändringar är inte tillåtna efter att slutskattebeskedet har skickats
        d) Kontakta Skatteverket via telefon
        """, "deklaration.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[8] = True
        else:
            answers[8] = False

        # Rättning
        correct = self.count_correct_answers(answers)

        print(f"-- Deklaration: \t{correct}/8 rätt")
        print()

    def evaluate_FMHCovid(self, model: Model) -> None:
        answers = {}

        # Fråga 1: Vad orsakar covid-19 och hur sprids sjukdomen?
        answer = model.getModelAnswer("""
                                      Vad orsakar covid-19 och hur sprids sjukdomen?
                                      a. Covid-19 orsakas av bakterier och sprids genom kontakt med smittade ytor.
                                      b. Covid-19 orsakas av ett virus och sprids genom luftvägsdroppar vid nära kontakt.
                                      c. Covid-19 orsakas av parasiter och sprids genom förorenat vatten.
                                      d. Covid-19 orsakas av svamp och sprids genom dammpartiklar i luften.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[1] = True
        else:
            answers[1] = False

        # Fråga 2: Vilka är de vanligaste symtomen på covid-19?
        answer = model.getModelAnswer("""
                                      Vilka är de vanligaste symtomen på covid-19?
                                      a. Hudutslag, tandvärk, håravfall.
                                      b. Snuva, feber, halsont, huvudvärk, förändrad smak och lukt, hosta.
                                      c. Yrsel, ledvärk, näsblod.
                                      d. Klåda, svullna fötter, buksmärtor.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[2] = True
        else:
            answers[2] = False

        # Fråga 3: Vad är en riskfaktor för att utveckla allvarlig covid-19?
        answer = model.getModelAnswer("""
                                      Vad är en riskfaktor för att utveckla allvarlig covid-19?
                                      a. Hög fysisk aktivitet.
                                      b. Hög ålder och annan sjuklighet.
                                      c. Regelbunden kontakt med pälsdjur vid allergi.
                                      d. Konsumption av alkohol och droger.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[3] = True
        else:
            answers[3] = False

        # Fråga 4: Hur lång är inkubationstiden för covid-19 sedan omikron-varianten tog över?
        answer = model.getModelAnswer("""
                                      Hur lång är inkubationstiden för covid-19 sedan omikron-varianten tog över?
                                      a. 1-2 dagar.
                                      b. 3-4 dagar.
                                      c. 7-10 dagar.
                                      d. 14-21 dagar.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False

        # Fråga 5: Vilken rekommendation ger Folkhälsomyndigheten till personer med ökad risk för allvarlig sjukdom?
        answer = model.getModelAnswer("""
                                      Vilken rekommendation ger Folkhälsomyndigheten till personer med ökad risk för allvarlig sjukdom?
                                      a. Att undvika former av kontakt med nya människor.
                                      b. Att regelbundet fylla på sitt vaccinskydd mot covid-19.
                                      c. Att använda naturläkemedel för att förebygga covid-19 med placebo.
                                      d. Att tvätta händerna för ofta i kallvatten utan tvål.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[5] = True
        else:
            answers[5] = False

        # Fråga 6: Vad är postcovid?
        answer = model.getModelAnswer("""
                                      Vad är postcovid?
                                      a. En ny typ av covid-19.
                                      b. Långvariga symtom efter en covid-19-infektion.
                                      c. Ett vaccin mot covid-19.
                                      d. Immunitet mot covid-19 smitta efter tillfriskning.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[6] = True
        else:
            answers[6] = False

        # Fråga 7: Vilken av följande spridningsvägar för covid-19 har visat sig ha en mycket liten roll?
        answer = model.getModelAnswer("""
                                      Vilken av följande spridningsvägar för covid-19 har visat sig ha en mycket liten roll?
                                      a. Nära kontakt mellan människor.
                                      b. Små och stora droppar från luftvägarna.
                                      c. Kontakt med förorenade ytor eller föremål.
                                      d. Inandning av luft i trånga utrymmen med bristande ventilation.
                                      """, "FHMCovid.pdf")

        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[7] = True
        else:
            answers[7] = False

        # Rättning
        correct = self.count_correct_answers(answers)

        print(f"-- Covid-19: \t\t{correct}/7 rätt")
        print()
    
    def evaluate_hovratt(self, model: Model) -> None:
        answers = {}

        # Fråga 1: Vilken lag reglerar ansvarsfördelningen vid sammanstötningar mellan fartyg?
        answer = model.getModelAnswer("""
                                      Vilken lag reglerar ansvarsfördelningen vid sammanstötningar mellan fartyg?
                                      a) Skadeståndslagen
                                      b) Sjölagen
                                      c) Trafikförordningen
                                      d) Försäkringslagen
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[1] = True
        else:
            answers[1] = False
        
        # Fråga 2: Vad var huvudsakliga orsaken till skadeståndsanspråket mot B.B.?
        answer = model.getModelAnswer("""
                                      Vad var huvudsakliga orsaken till skadeståndsanspråket mot B.B.?
                                      a) B.B. hade inte tillräcklig bemanning ombord.
                                      b) B.B. hade inte betalat sina avgifter.
                                      c) B.B. använde ett felaktigt gasreglage.
                                      d) B.B. körde båten för snabbt.
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: a)")
        if answer == "a":
            answers[2] = True
        else:
            answers[2] = False
        
        # Fråga 3: Vilken säkerhetsanordning saknades som kunde ha förhindrat olyckan?
        answer = model.getModelAnswer("""
                                      Vilken säkerhetsanordning saknades som kunde ha förhindrat olyckan?
                                      a) Livbåtar
                                      b) Brandsläckare
                                      c) Dödsmansgrepp
                                      d) Radar
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: c)")
        if answer == "c":
            answers[3] = True
        else:
            answers[3] = False
        
        # Fråga 4: Vad hävdade dödsboet som orsak till sammanstötningen?
        answer = model.getModelAnswer("""
                                      Vad hävdade dödsboet som orsak till sammanstötningen?
                                      a) B.B. var oaktsam.
                                      b) Sammanstötningen var en olyckshändelse.
                                      c) B.B. följde inte sjövägsreglerna.
                                      d) Båten var felaktigt konstruerad.
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[4] = True
        else:
            answers[4] = False
        
        # Fråga 5: Vilken var hovrättens bedömning av B.B:s agerande vid slussningen?
        answer = model.getModelAnswer("""
                                      Vilken var hovrättens bedömning av B.B:s agerande vid slussningen?
                                      a) B.B. agerade korrekt enligt alla föreskrifter.
                                      b) B.B. var vårdslös och följde inte säkerhetsföreskrifterna.
                                      c) B.B. hade ingen kontroll över båten.
                                      d) B.B. följde alla säkerhetsföreskrifter men hade otur.
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[5] = True
        else:
            answers[5] = False
        
        # Fråga 6: Vilken lag föreskriver att ett fartyg ska vara sjövärdigt och bemannat på ett betryggande sätt?
        answer = model.getModelAnswer("""
                                      Vilken lag föreskriver att ett fartyg ska vara sjövärdigt och bemannat på ett betryggande sätt?
                                      a) Sjötrafikförordningen
                                      b) Sjölagen
                                      c) Skadeståndslagen
                                      d) Försäkringslagen
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[6] = True
        else:
            answers[6] = False
        
        # Fråga 7: Vilken presumtion finns i sjörättspraxis vid sammanstötningar mellan fartyg?
        answer = model.getModelAnswer("""
                                      Vilken presumtion finns i sjörättspraxis vid sammanstötningar mellan fartyg?
                                      a) Att felaktig manöver alltid orsakas av tekniska fel.
                                      b) Att felaktig manöver har orsakats av oaktsamhet ombord.
                                      c) Att sammanstötningen är alltid på grund av dåligt väder.
                                      d) Att ingen är ansvarig vid sammanstötning.
                                      """, "hovrätt.pdf")
        
        if model.printAnswers: print(answer, "(rätt: b)")
        if answer == "b":
            answers[7] = True
        else:
            answers[7] = False
        
        # Rättning
        correct = self.count_correct_answers(answers)
        
        print(f"-- Hovrätt: \t\t{correct}/7 rätt")
        print()


In [20]:
from groq import Groq

class GroqModel(Model):

    def setupModel(self):
        load_dotenv()
    
        GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    
        client = Groq(
            api_key=GROQ_API_KEY
        )
    
        self.client = client

    def getModelAnswer(self, prompt, doc):

        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    'content': self.systemPrompt
                    
                },
                {
                    "role": "user",
                    "content": self.extract_text_from_pdf(doc) + "\n" + prompt + "\n" + "Glöm ej att endast svara med bokstaven som motsvarar det korrekta svaret.",
                }
            ],
            model="llama3-70b-8192",
        )
    
        return chat_completion.choices[0].message.content

In [21]:
def main():
    print("# Evaluating Groq model:")
    groq = GroqModel(printAnswers=True)
    groqEvaluator = Evaluator(groq)
    groqEvaluator.evaluate_all(groq)
    #groqEvaluator.evaluate_hyresavi(groq)
    #groqEvaluator.evaluate_hyresavi2(groq)
    #groqEvaluator.evaluate_krisbrochure(groq)
    #groqEvaluator.evaluate_Askatt(groq)
    #groqEvaluator.evaluate_deklaration(groq)
    #groqEvaluator.evaluate_FMHCovid(groq)
    #groqEvaluator.evaluate_hovratt(groq)

main()

# Evaluating Groq model:
c (rätt: c)
e (rätt: f)
c (rätt: c)
b (rätt: b)
-- Hyresavi: 		3/4 rätt

a (rätt: a)
c (rätt: c)
d (rätt: b)
b (rätt: b)
a (rätt: a)
a (rätt: a)
c (rätt: c)
b (rätt: b)
-- Hyresavi 2: 		7/8 rätt

c (rätt: c)
b (rätt: b)
a (rätt: a)
b (rätt: b)
b (rätt: b)
c (rätt: c)
b (rätt: b)
a (rätt: a)
-- Krisbrochure: 		8/8 rätt

b (rätt: b)
a (rätt: a)
b (rätt: d)
a (rätt: a)
a (rätt: d)
-- A-skatt: 		3/5 rätt

a (rätt: a)
d (rätt: d)
b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
-- Deklaration: 		8/8 rätt

b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
b (rätt: b)
c (rätt: c)
-- Covid-19: 		7/7 rätt



RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j0jdy2prf2ns6nmw8y1sepyd` on tokens per minute (TPM): Limit 6000, Used 6380, Requested 5652. Please try again in 1m0.327s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

# Evaluating Groq model:
-- Hyresavi: 3/4 rätt

-- Hyresavi 2: 7/8 rätt

-- Krisbrochure: 8/8 rätt

-- A-skatt: 3/5 rätt

-- Deklaration: 8/8 rätt

-- Covid-19 dokument: 7/7 rätt

-- Hovrätt: 7/7 rätt